In [ ]:
#!pip install --upgrade pip
#!pip install beautifulsoup4
#!pip install feedparser

## wikipedia 한국어 

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
from urllib import parse

def loadPage(baseUrl, path, targetDir, depth, indexWriter, linkset):
    if path in linkset :
        return
    
    linkset.add(path)
    
    url = baseUrl + path
    print("loading ", url)
    
    try :
        html = urlopen(url)
        bsObj = BeautifulSoup(html, "html.parser")
        title = bsObj.find("h1", {"id":"firstHeading"}).text

        contentsDiv = bsObj.find("div", {"id":"mw-content-text"})
        contentsDiv = contentsDiv.find("div", {"class":"mw-parser-output"})

        links = []

        for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", 
            href=re.compile("^(/wiki/)((?!:).)*$")):
              if 'href' in link.attrs:
                  links.append(link.attrs['href'])

        filename = parse.quote(title)
        filename = filename.replace('/','_') + ".txt"
        filepath = targetDir + "/" + filename

        indexWriter.writerow([title, filename])

        with open(filepath, 'w', encoding='utf-8') as file :
            file.write(title)
            file.write("\n\n")
            file.write(contentsDiv.text)
            file.close()

        if(depth > 0):
            depth -= 1
            for link in links:
                loadPage(baseUrl, link, targetDir, depth, indexWriter, linkset)
    except Exception as ex :  
        print(ex)
    
        

# 경제사상사 : /wiki/%EA%B2%BD%EC%A0%9C%EC%82%AC%EC%83%81%EC%82%AC
economicThoughts = "/wiki/%EA%B2%BD%EC%A0%9C%EC%82%AC%EC%83%81%EC%82%AC"
# 경제학 : /wiki/%EA%B2%BD%EC%A0%9C%ED%95%99
economics = "/wiki/%EA%B2%BD%EC%A0%9C%ED%95%99"
# 은닉마르코프 모델 : /wiki/%EC%9D%80%EB%8B%89_%EB%A7%88%EB%A5%B4%EC%BD%94%ED%94%84_%EB%AA%A8%EB%8D%B8
markov = "/wiki/%EC%9D%80%EB%8B%89_%EB%A7%88%EB%A5%B4%EC%BD%94%ED%94%84_%EB%AA%A8%EB%8D%B8"
# 체계이론 : /wiki/%EB%B6%84%EB%A5%98:%EC%B2%B4%EA%B3%84%EC%9D%B4%EB%A1%A0
system = "/wiki/%EB%B6%84%EB%A5%98:%EC%B2%B4%EA%B3%84%EC%9D%B4%EB%A1%A0"
with open("../data/wiki_page_index.csv", 'w', encoding='utf-8') as file :
    indexWriter = csv.writer(file)
    linkset = set()
    loadPage("https://ko.wikipedia.org", system, 
             "../../data/wiki", 2, indexWriter, linkset)
    file.close()

# CIOBIZ

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
from urllib import parse

host_url = "http://www.ciobiz.co.kr"
article_path = "/news/article.html?id="
article_path_len = len(article_path)
file_dir = "../../data/ciobiz/"


def loadList(page, idset):
    html = urlopen("http://www.ciobiz.co.kr/news/section.html?id1=01&page="+str(page))
    bsObj = BeautifulSoup(html, "html.parser")
    listDiv = bsObj.find("ul", {"class":"list_news"})
    for li in listDiv.findAll("li"):
        href = li.find("dt").find("a").attrs['href']
        id = href[article_path_len:]
        idset.add(id)

def loadNews(url, filepath):
    print("Loading " , url)
    html = urlopen(url)
    bsObj = BeautifulSoup(html, "html.parser")
    containerDiv = bsObj.find("div", {"id":"container"})
    article = containerDiv.find("article")
    title = article.find("h2", {"class":"article_title"}).text
    bodySect = containerDiv.find("section", {"class":"article_body"})
    body = bodySect.find("p")
    
    with open(filepath, 'w', encoding='utf-8') as file :
        file.write(url)
        file.write("\n")
        file.write(title)
        file.write("\n")
        file.write(body.text)
        file.close()
            
idset = set()
for page in range(5):
    loadList(page,idset)

print(len(idset))

for id in idset:
    new_url = host_url + article_path + id
    filepath = file_dir + id + ".txt"
    
    try:
        loadNews(new_url, filepath)
    except Exception as e:
        print (e)

# ZDNet korea

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import csv
from urllib import parse

host_url = "http://www.zdnet.co.kr"
list_path = "/news/news_keyword.asp?keyword=" + parse.quote("인공지능") + "&page="
article_path = "/news/news_view.asp?artice_id="
article_path_len = len(article_path)
file_dir = "../../data/zdnet/"


def loadList(page, idset):
    url = host_url + list_path + str(page)
    print(url)
    html = urlopen(Request(url, headers={'User-Agent': 'Mozilla'}))
    bsObj = BeautifulSoup(html, "html.parser")

    for li in bsObj.findAll("div", {"class":"article_li"}):
        href = li.find("h2").find("a").attrs['href']
        id = href[article_path_len:]
        idset.add(id)

def loadNews(url, filepath):
    print("Loading " , url)
    html = urlopen(Request(url, headers={'User-Agent': 'Mozilla'}))
    bsObj = BeautifulSoup(html, "html.parser")
    
    title = bsObj.find("div", {"class":"sub_view_tit2"}).text
    contents = bsObj.find("div", {"id":"content"})
    lines = contents.findAll("p")
    
    with open(filepath, 'w', encoding='utf-8') as file :
        try:
            file.write(url)
            file.write("\n")
            file.write(title)
            for line in lines:
                file.write("\n")
                file.write(line.text)
        except Exception as e:
            print(e)
        file.close()
            
idset = set()
for page in range(10):
    loadList(page+1,idset)

print(len(idset))

for id in idset:
    new_url = host_url + article_path + id
    filepath = file_dir + id + ".txt"
    
    try:
        loadNews(new_url, filepath)
    except Exception as e:
        print (e)

# Blotter

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import csv
from urllib import parse

host_url = "http://www.bloter.net"
list_path = "/archives/category/society/page/"
article_path = "/archives/"
article_path_len = len("article-")
file_dir = "../../data/blotter/"


def loadList(page, idset):
    url = host_url + list_path + str(page)
    print(url)
    html = urlopen(Request(url, headers={'User-Agent': 'Mozilla'}))
    bsObj = BeautifulSoup(html, "html.parser")
    body = bsObj.find("div", {"class":"category--body--main"})

    for li in body.findAll("article"):
        nodeid = li.attrs['id']
        id = nodeid[article_path_len:]
        idset.add(id)

def loadNews(url, filepath):
    print("Loading " , url)
    html = urlopen(Request(url, headers={'User-Agent': 'Mozilla'}))
    bsObj = BeautifulSoup(html, "html.parser")
    
    title = bsObj.find("h1", {"class":"headline"}).text
    contents = bsObj.find("div", {"class":"article--content"})
    lines = contents.findAll("p")
    
    with open(filepath, 'w', encoding='utf-8') as file :
        try:
            file.write(url)
            file.write("\n")
            file.write(title)
            for line in lines:
                file.write("\n")
                file.write(line.text)
        except Exception as e:
            print(e)
        file.close()
            
idset = set()
for page in range(10):
    loadList(page+1,idset)

print(len(idset))

for id in idset:
    new_url = host_url + article_path + id
    filepath = file_dir + id + ".txt"
    
    try:
        loadNews(new_url, filepath)
    except Exception as e:
        print (e)

In [1]:
#!pip install feedparser

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import os
import csv
from urllib import parse

import feedparser

def handleKhan(bsObj):
    ttlObj = bsObj.find("h1",{"id":"article_title"})
    title = ttlObj.text if ttlObj else ''
    contents = bsObj.find("div",{"class":"art_body"}).text
    return title, contents


def handleJoins(bsObj):
    title = bsObj.find("h1",{"id":"article_title"}).text
    contents = bsObj.find("div",{"id":"article_body"}).text
    return title, contents


def handleHani(bsObj):
    title = bsObj.find("span",{"class":"title"}).text
    contents = bsObj.find("div",{"class":"article-text"}).find("div",{"class":"text"}).text
    return title, contents


def handleNocut(bsObj):
    title = bsObj.find("div", {"id":"pnlViewTop"}).find("h2").text
    contents = bsObj.find("div",{"id":"pnlViewBox"}).text
    return title, contents


rssfeed_map = {'hani':'http://www.hani.co.kr/rss/',
                'joins':'https://rss.joins.com/joins_news_list.xml',
               'khan':'http://www.khan.co.kr/rss/rssdata/total_news.xml', 
                'nocut':'http://rss.nocutnews.co.kr/nocutnews.xml'}

handler_map = {'hani':handleHani,
                'joins':handleJoins,
               'khan':handleKhan,
                'nocut':handleNocut}

newsdir = "../../data/news"


def loadpage(url, filepath, dochandler):
    try :
        html = urlopen(Request(url, headers={'User-Agent': 'Mozilla'}))
        bsObj = BeautifulSoup(html, "html.parser")
        title, contents = dochandler(bsObj)

        with open(filepath, 'w', encoding='utf-8') as file :
            file.write(url)
            file.write("\n")
            file.write(title)
            file.write("\n")
            file.write(contents)
            file.close()
    except Exception as ex :  
        print(url,ex)
    
for name, url in rssfeed_map.items():
    feed = feedparser.parse(url)
    print(feed['feed']['title'])
    length = len(feed['entries'])
    
    handler = handler_map[name]
    
    for index in range(length):
        newsurl = feed.entries[index]['link']
#         print(newsurl)
        filepath = os.path.join(newsdir,name+"-"+str(index) + ".txt")
        loadpage(newsurl, filepath, handler)
        print('.', end="")

전체기사 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
..............................중앙일보 | 전체기사
..............................경향신문:전체기사
.....................................................................................http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201808241518001&code=910303&utm_campaign=rss_btn_click&utm_source=khan_rss&utm_medium=rss&utm_content=total_news 'NoneType' object has no attribute 'text'
............................................................http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201808240000001&code=940100&utm_campaign=rss_btn_click&utm_source=khan_rss&utm_medium=rss&utm_content=total_news 'NoneType' object has no attribute 'text'
..............................................................................................................................................................................http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201808231106001&code=940100&utm_campaign=rss_btn_click&utm_source=khan_rss&utm_medium=rss&u